In [21]:
import sys

# So it can see the scripts folder
sys.path.append("../")


In [22]:
import nest_asyncio
from datasets import Dataset
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.evolutions import multi_context, reasoning, simple
from ragas.testset.generator import TestsetGenerator

from scripts.constants import GPT3_MODEL, GPT4_MODEL, OPENAI_EMBEDDINGS_MODEL
from scripts.parse_json import PDFJson
from llama_index.core import Document


load_dotenv()

nest_asyncio.apply()

generator_llm = ChatOpenAI(model=GPT3_MODEL)
critic_llm = ChatOpenAI(model=GPT4_MODEL)
embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDINGS_MODEL)

In [24]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings,
)

distributions = {simple: 0.33, multi_context: 0.33, reasoning: 0.33}



In [26]:
pdf_json = PDFJson.from_json_file(
    "../parse-output/2302.09057.json"
)
pdf_md = pdf_json.full_md
index_docs = [
    Document(
        metadata={"pdf_path": pdf_json.file_path, "pdf_id": pdf_json.job_id},
        text=pdf_md,
    )
]

In [28]:
testset = generator.generate_with_llamaindex_docs(
    index_docs,
    test_size=3,
    distributions=distributions,
    with_debugging_logs=True,
)

KeyboardInterrupt: 